# CSE-CIC-IDS 2017

In [1]:
model_id = "decisiontree1-multiclass"

In [2]:
import numpy as np
np.random.seed(42)
np.set_printoptions(suppress=True)
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

import glob, pickle, time, sklearn

In [3]:
print(sklearn.__version__)

0.21.3


In [4]:
NOTEBOOK_PATH = "C:/Users/Xetrov/Desktop/SciFair20/Code/"

In [5]:
x_scaled = pd.read_csv(NOTEBOOK_PATH + "IDS2017/x_scaled_powertransform.csv")

y_df_enc = pd.read_csv(NOTEBOOK_PATH + "IDS2017/y_grouped_1henc.csv")

# Split data into train and test

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
x_train, x_valtest, y_train, y_valtest = train_test_split(x_scaled, y_df_enc, test_size = 0.4, random_state = 42)

In [8]:
x_val, x_test, y_val, y_test = train_test_split(x_valtest, y_valtest, test_size = 0.5, random_state = 42)

In [9]:
del x_train
del y_train

del x_valtest 
del y_valtest

# ADASYN

In [10]:
x_train_res = pd.read_csv(NOTEBOOK_PATH + "IDS2017/x_adasyn.csv").to_numpy()

In [11]:
y_train_res = pd.read_csv(NOTEBOOK_PATH + "IDS2017/y_adasyn.csv")

In [12]:
y_train_res = y_train_res[y_train_res.columns[0]].to_numpy()

# Train model

In [13]:
# from sklearn.model_selection import GridSearchCV
# from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

In [15]:
clf = DecisionTreeClassifier(max_depth=None, random_state=42)

In [16]:
%%time
clf.fit(x_train_res, y_train_res)

Wall time: 7min 17s


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=42, splitter='best')

**Save model**

In [17]:
import pickle

In [18]:
import time

model_filename = model_id  + " [" + time.strftime("%Y%m%d %H%M") + "]"

In [19]:
save_file = open(NOTEBOOK_PATH + "Models/" + model_filename + ".pkl", "wb")
saved_model = pickle.dump(clf, save_file)
save_file.close()

**Test model**

In [20]:
pred = clf.predict(x_val)

In [21]:
pred_index = pred
y_index = list(np.argmax(y_val.to_numpy(), axis=1))

atktypes = sorted(y_val.columns)
pred_series = pd.Series(pred_index, name="Pred").replace(dict(enumerate(atktypes)))
y_series = pd.Series(y_index, name="Actual").replace(dict(enumerate(atktypes)))

matrix = pd.crosstab(pred_series, y_series, margins=True)
matrix

Actual,BENIGN,Botnet,Brute Force,DoS/DDoS,Infiltration,PortScan,Web Attack,All
Pred,,,,,,,,
BENIGN,453723,34,0,36,0,188,0,453981
Botnet,167,363,0,0,0,0,0,530
Brute Force,1,0,2809,0,0,0,0,2810
DoS/DDoS,113,0,0,76464,0,3,11,76591
Infiltration,1,0,0,0,7,0,0,8
PortScan,198,0,1,10,0,31593,10,31812
Web Attack,4,0,0,0,0,3,410,417
All,454207,397,2810,76510,7,31787,431,566149


# F1 Score

In [22]:
from sklearn.metrics import f1_score, precision_score, recall_score

In [23]:
pred = clf.predict(x_test)
pred_index = list(pred)

In [24]:
y_index = list(np.argmax(y_test.to_numpy(), axis=1))
f1_micro = f1_score(y_index, pred_index, average='micro')
f1_macro = f1_score(y_index, pred_index, average='macro')

print(f1_micro)
print(f1_macro)

0.9986628961633774
0.942897100630209


In [25]:
atktypes = sorted(y_val.columns)
pred_series = pd.Series(pred_index, name="Pred").replace(dict(enumerate(atktypes)))
y_series = pd.Series(y_index, name="Actual").replace(dict(enumerate(atktypes)))

matrix = pd.crosstab(pred_series, y_series, margins=True)
# matrix.to_csv(NOTEBOOK_PATH + "Confusion Matrices/test_" + model_filename + ".csv")
matrix

Actual,BENIGN,Botnet,Brute Force,DoS/DDoS,Infiltration,PortScan,Web Attack,All
Pred,,,,,,,,
BENIGN,454333,36,3,32,0,179,0,454583
Botnet,120,348,0,0,0,0,0,468
Brute Force,0,0,2749,0,0,0,0,2749
DoS/DDoS,119,0,0,75843,0,3,16,75981
Infiltration,2,0,0,0,5,0,0,7
PortScan,210,0,0,17,0,31691,6,31924
Web Attack,6,0,0,5,0,3,423,437
All,454790,384,2752,75897,5,31876,445,566149
